# Results processing and data viz
## From microsimulation experiments

It includes some preeliminary plots, the summary plots, and the creation of grid maps and animations

In [ ]:
#imports
import os, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm
import matplotlib.colors as colors
plt.rcParams.update({'font.size': 16})
pd.set_option('display.max_rows', 3000)

#Make sure that the SUMO_HOME environment variable is correctly set after installation:
SUMO_HOME = r"C:\Program Files (x86)\Eclipse\Sumo"

## Source results files

In [ ]:
results_folder_net01 = r"net01/data/results/"
results_folder_net02 = r"net02/data/results/"
results_folder_net03 = r"net03/data/results/"
results_folder_net04 = r"net04/data/results/"

## Traffic fundamental diagram
Network metrics are needed for normalizing the results accross the different networks.

In [ ]:
#Total length of lanes for the different networks
#These values are obtained manually from the nets, using SUMO-GUI and NETEDIT
net_length_net01 = 53920

net_length_net02_x2lanes = 2456.6 + 2516.6 + 2943.12 + 929.84 + 955.44 + 506.52 + 5373.72 + 4629.2 + 2854.32 + 1038.64 + 1503.96 + 507.32 + 504.52
net_length_net02_x1lanes = 504.32 + 979.04 + 475.72 + 490.52 + 754.98 + 965.24 + 1904.88 + 1851.68 + 978.04 + 964.24
net_length_net02 = net_length_net02_x2lanes + net_length_net02_x1lanes

net_length_net03_x2lanes = 21186.08
net_length_net03_x1lanes = 7263.8 + 5811.04
net_length_net03 = net_length_net03_x2lanes + net_length_net03_x1lanes

net_length_net04 = 58918.52


# Num of edges per network
num_edges_net01 = 220
num_edges_net02 = 192
num_edges_net03 = 196
num_edges_net04 = 242

Helper functions for transforming the original XML files that SUMO creates as results of the simulation into CSV files and then dataframes.

In [ ]:
# Helper function for transforming XLM output files into 

def transform_aadt_output_xml_to_csv(results_folder):
    if not os.path.exists(results_folder + r"/csv"):
        os.makedirs(results_folder + r"/csv")

    for file in os.listdir(results_folder):
        if file.endswith("aadt_output_freq60s.xml"):
            input_xml_file = str(results_folder + file)
            output_csv_file = str(results_folder + r"/csv/" + file[:-3] + "csv")
            # bash command
            !python "$SUMO_HOME\tools\xml\xml2csv.py" \
            --output "{output_csv_file}" \
            "{input_xml_file}"

# Building new pandas dataframe based on summary data from the results summary csv files

def create_df_for_traffic_fundamental_diagram(results_folder, net_length, num_edges):
    
    column_names = ['experiment', 'scale', 'lanes_length_m', 'flow', 'density', 'freq', 'edge_entered', 'edge_departed', 'edge_sampledSeconds']
    df_output = pd.DataFrame(columns = column_names)
    #iterate through the folder
    for file in os.listdir(results_folder + "/csv/"):
        if file.endswith("aadt_output_freq60s.csv"):
            output_csv_file =  str(results_folder + r"/csv/" + file)
            scale = float(output_csv_file.split('_')[-5])
            experiment = results_folder.split('/')[0]
            freq = 60
            #transform to pandas df
            df = pd.read_csv(output_csv_file, sep=";")
            #groupby
            df_g = df.groupby('interval_begin')[['edge_entered', 'edge_departed', 'edge_sampledSeconds']].apply(sum)
            #create variables of fundamental traffic diagram (#some refs: https://www.eclipse.org/lists/sumo-user/msg06970.html)
            df_g['scale'] = scale
            df_g['lanes_length_m'] = net_length
            df_g['flow'] = (((df_g.edge_entered + df_g.edge_departed)/freq)*3600)/num_edges #maybe freq here no TODO!!!!!
            df_g['density'] = (df_g.edge_sampledSeconds/freq)/(net_length/1000)
            df_g['experiment'] = experiment
            df_g['freq'] = freq
            #merge
            frames = [df_output, df_g]
            df_output = pd.concat(frames)
    
    return df_output

Helper functions for smoothing the data and obtaining the critical points of the fundamental diagram.

In [ ]:
from scipy.interpolate import pchip
import statsmodels.api as sm
from operator import itemgetter

# COMBINING PCHIP AND LOWESS
def fit(x,y):
    
    pch = pchip(x, y)
    
    xx = np.linspace(x[0], x[-1], 1000)
    yy = pch(xx)
    
    lowess = sm.nonparametric.lowess(yy, xx, frac=0.2)
    
    x_lowess = lowess[:, 0]
    y_lowess = lowess[:, 1]
    return [x_lowess, y_lowess]


def estimate_critical_metrics(exp_df):
    
    exp_df_mod = exp_df.groupby('density').mean()
    exp_df_mod = exp_df_mod.sort_index()
    
    x = exp_df_mod.index
    y = exp_df_mod.flow

    [x2, y2] = fit(x, y)
    
    #for avoiding errors in high values in congested phase
    _x2 = []
    _y2 = []
    for i in range(len(x2)):
        if x2[i] < 80:
            _x2.append(x2[i])
            _y2.append(y2[i])
    _x2 = np.array(_x2)
    _y2 = np.array(_y2)
        
    max_flow = max(_y2)
    density_at_max_flow = _x2[_y2.argmax()]
    estimated_optimal_speed = (max_flow/density_at_max_flow) 
    return (max_flow, density_at_max_flow, estimated_optimal_speed, x2, y2)

def create_df_critical_metrics():
    
    column_names = ['experiment', 'lanes_length_m', 'max_flow', 'density_At_max_flow', 'optim_speed']
    df_compa = pd.DataFrame(columns = column_names)
    
    for df_exp in [df_tdf_002, df_tdf_032, df_tdf_052, df_tdf_102]:
        critical_metrics = estimate_critical_metrics(df_exp)
        df_compa.loc[len(df_compa.index)] = [df_exp.experiment.iloc[0], 
                                             df_exp.lanes_length_m.iloc[0], 
                                             critical_metrics[0], 
                                             critical_metrics[1], 
                                             critical_metrics[2]]
    
    return df_compa

In [ ]:
transform_aadt_output_xml_to_csv(results_folder_net01)
transform_aadt_output_xml_to_csv(results_folder_net02)
transform_aadt_output_xml_to_csv(results_folder_net03)
transform_aadt_output_xml_to_csv(results_folder_net04)

df_tdf_002 = create_df_for_traffic_fundamental_diagram(results_folder_net01, net_length_net01, num_edges_net01)
df_tdf_032 = create_df_for_traffic_fundamental_diagram(results_folder_net02, net_length_net02, num_edges_net02)
df_tdf_052 = create_df_for_traffic_fundamental_diagram(results_folder_net03, net_length_net03, num_edges_net03)
df_tdf_102 = create_df_for_traffic_fundamental_diagram(results_folder_net04, net_length_net04, num_edges_net04)

In [ ]:
df_critical_metrics = create_df_critical_metrics()
df_critical_metrics

## Viz
Basic fundamental diagrams comparison

In [ ]:
# Make directory for comparative fundamental diagrams
if not os.path.exists(r"charts"):
    os.makedirs(r"charts")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,15), sharex=True, 
                       gridspec_kw={'hspace': 0, 'wspace': 0})

fig.suptitle('Fundamental diagram of traffic flow', fontsize=16, y=0.9)
col = ["#648FFF", "#DC267F", "#FE0000", "#FFB000"]
# col = ["#9b7000", "#23d980", "#01ffff", "#004fff"] #INVERT

ax.scatter(df_tdf_002.density, df_tdf_002['flow'], color=col[0], s=50, alpha=0.05)
ax.scatter(df_tdf_032.density, df_tdf_032['flow'], color=col[1], s=50, alpha=0.05)
ax.scatter(df_tdf_052.density, df_tdf_052['flow'], color=col[2], s=50, alpha=0.05)
ax.scatter(df_tdf_102.density, df_tdf_102['flow'], color=col[3], s=50, alpha=0.05)

ax.plot(estimate_critical_metrics(df_tdf_002)[3], estimate_critical_metrics(df_tdf_002)[4], color=col[0], label="net01 (existing situation)")
ax.plot(estimate_critical_metrics(df_tdf_032)[3], estimate_critical_metrics(df_tdf_032)[4], color=col[1], label="net02 (Cerda's original grid)")
ax.plot(estimate_critical_metrics(df_tdf_052)[3], estimate_critical_metrics(df_tdf_052)[4], color=col[2], label="net03 (Superblocks grid)")
ax.plot(estimate_critical_metrics(df_tdf_102)[3], estimate_critical_metrics(df_tdf_102)[4], color=col[3], label="net04 (w/ diagonal avenue)")

ax.set_ylabel('Flow (veh/h) - per edge of the network')
ax.set_xlabel('Density (veh/km)')
ax.legend(loc="upper right")
ax.set_xlim(-5,140)
ax.set_ylim(-50,1400)
ax.axhline(y=0, xmin=-0.05, xmax=0.1, color=(0.25, 0.25, 0.25, 0.25))
ax.axvline(x=0, ymin=-0.05, ymax=0.1, color=(0.25, 0.25, 0.25, 0.25))

fig.savefig("charts/MFD01_compar_01-02-03-04_micro.png",
        orientation='portrait',
        transparent=True, bbox_inches='tight', pad_inches=0,
        frameon=None, metadata=None)

Fundamental diagram comparison with scale values for each simulation

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(25,15), sharex=True, 
                       gridspec_kw={'hspace': 0, 'wspace': 0})

color_002 = cm.get_cmap('coolwarm')

fig.suptitle('Fundamental diagram of traffic flow with scale', fontsize=16, y=0.9)

ax.scatter(df_tdf_002.density, df_tdf_002['flow'], marker='o', s=100, alpha=0.1, color=color_002(df_tdf_002.scale/2), label="net01 (existing situation). Sampling points")
ax.scatter(df_tdf_032.density, df_tdf_032['flow'], marker='+', s=100, alpha=0.25, color=color_002(df_tdf_032.scale/2), label="net02 (Cerda's original grid). Sampling points")
ax.scatter(df_tdf_052.density, df_tdf_052['flow'], marker='^', s=100, alpha=0.25, color=color_002(df_tdf_052.scale/2), label="net03 (Superblocks grid). Sampling points")
ax.scatter(df_tdf_102.density, df_tdf_102['flow'], marker='x', s=100, alpha=0.1, color=color_002(df_tdf_102.scale/2), label="net04 (w/ diagonal avenue). Sampling points")

ax.plot(estimate_critical_metrics(df_tdf_002)[3], estimate_critical_metrics(df_tdf_002)[4], color=col[0], lw=3, label="net01 (existing situation). Trend line")
ax.plot(estimate_critical_metrics(df_tdf_032)[3], estimate_critical_metrics(df_tdf_032)[4], color=col[1], lw=3, label="net02 (Cerda's original grid). Trend line")
ax.plot(estimate_critical_metrics(df_tdf_052)[3], estimate_critical_metrics(df_tdf_052)[4], color=col[2], lw=3, label="net03 (Superblocks grid). Trend line")
ax.plot(estimate_critical_metrics(df_tdf_102)[3], estimate_critical_metrics(df_tdf_102)[4], color=col[3], lw=3, label="net04 (w/ diagonal avenue). Trend line")

ax.set_ylabel('Flow (veh/s)')
ax.set_xlabel('Density (veh/km)')
ax.legend(loc="upper right")
ax.set_xlim(-5,140)
ax.set_ylim(-50,1400)
ax.axhline(y=0, xmin=-0.05, xmax=0.1, color=(0.25, 0.25, 0.25, 0.25))
ax.axvline(x=0, ymin=-0.05, ymax=0.1, color=(0.25, 0.25, 0.25, 0.25))

cNorm  = colors.Normalize(vmin=0, vmax=2)
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=color_002)
scalarMap.set_array([])
cbar = fig.colorbar(scalarMap,ax=ax, aspect=40)
cbar.set_label('scale of demand load (1 = current adjusted demand)', rotation=270,  labelpad=-70)

fig.savefig("charts/MFD02_compar_01-02-03-04_micro.png", dpi=None,
        orientation='portrait', papertype=None, format=None,
        transparent=True, bbox_inches='tight', pad_inches=0,
        frameon=None, metadata=None)